In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [50]:
data= pd.read_csv('Churn_Modelling.csv')
data.head

<bound method NDFrame.head of       RowNumber  CustomerId    Surname  ...  IsActiveMember EstimatedSalary Exited
0             1    15634602   Hargrave  ...               1       101348.88      1
1             2    15647311       Hill  ...               1       112542.58      0
2             3    15619304       Onio  ...               0       113931.57      1
3             4    15701354       Boni  ...               0        93826.63      0
4             5    15737888   Mitchell  ...               1        79084.10      0
...         ...         ...        ...  ...             ...             ...    ...
9995       9996    15606229   Obijiaku  ...               0        96270.64      0
9996       9997    15569892  Johnstone  ...               1       101699.77      0
9997       9998    15584532        Liu  ...               1        42085.58      1
9998       9999    15682355  Sabbatini  ...               0        92888.52      1
9999      10000    15628319     Walker  ...              

In [51]:
# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [52]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [53]:
# One-hot encode 'Geography'
one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [54]:
# Combine the encoded columns with the original data
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [55]:
# Split the data into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

## Split the features and target variable into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
# Save the encoders and scalers for future use

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Problem Statement

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [58]:
# Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression. Linear activation is default for regression tasks
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae']) # Compile the model with MAE loss for regression

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard logging
log_dir = "regressions/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [60]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [61]:
# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
200/200 [==============================] - 2s 7ms/step - loss: 100000.6719 - mae: 100000.6719 - val_loss: 102116.1328 - val_mae: 102116.1328
Epoch 2/100
200/200 [==============================] - 1s 7ms/step - loss: 99938.3203 - mae: 99938.3203 - val_loss: 101995.7578 - val_mae: 101995.7578
Epoch 3/100
200/200 [==============================] - 2s 8ms/step - loss: 99751.4609 - mae: 99751.4609 - val_loss: 101753.7109 - val_mae: 101753.7109
Epoch 4/100
200/200 [==============================] - 1s 7ms/step - loss: 99445.0391 - mae: 99445.0391 - val_loss: 101419.3281 - val_mae: 101419.3281
Epoch 5/100
200/200 [==============================] - 1s 7ms/step - loss: 99055.5391 - mae: 99055.5391 - val_loss: 101015.8594 - val_mae: 101015.8594
Epoch 6/100
200/200 [==============================] - 2s 8ms/step - loss: 98612.4297 - mae: 98612.4297 - val_loss: 100568.9062 - val_mae: 100568.9062
Epoch 7/100
200/200 [==============================] - 1s 7ms/step - loss: 98167.9297 - mae:

In [62]:
%load_ext tensorboard
%tensorboard --logdir regressions/logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 80710), started 0:18:53 ago. (Use '!kill 80710' to kill it.)

In [63]:
# Alternative: Open TensorBoard in browser
import webbrowser
import time

print("TensorBoard should be accessible at: http://localhost:6007")
print("If the iframe above doesn't work, click this link or copy-paste the URL into your browser:")
print("http://localhost:6007")

# Optionally, automatically open in browser
# webbrowser.open('http://localhost:6007')

# Check if TensorBoard is running
import subprocess
try:
    result = subprocess.run(['lsof', '-i', ':6007'], capture_output=True, text=True)
    if result.stdout:
        print("\n✅ TensorBoard is running on port 6007")
    else:
        print("\n❌ TensorBoard might not be running on port 6007")
except:
    print("\nCouldn't check port status, but TensorBoard should be running")

TensorBoard should be accessible at: http://localhost:6007
If the iframe above doesn't work, click this link or copy-paste the URL into your browser:
http://localhost:6007

✅ TensorBoard is running on port 6007


In [64]:
## Evaluate the model
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 [==============================] - 0s 5ms/step - loss: 56339.1992 - mae: 56339.1992
Test Loss: 56339.19921875, Test MAE: 56339.19921875


In [65]:
model.save('salary_regression_model.h5')

/Users/a825765/Documents/me/NUS/Learn/Machine Learning/ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
